    Import các thư viện cần thiết

In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from tqdm import tqdm
import csv
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import unidecode
from cmath import nan
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , RobustScaler, StandardScaler
import warnings
import re
import threading
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os
warnings.filterwarnings("ignore")

    Crawl data

    URL = view-source:https://batdongsan.com.vn

Khởi tạo đối tượng webdriver và khai báo link URL

In [2]:
#URL = view-source:https://batdongsan.com.vn
productLinks = []

Lấy tất cả các link của bất động sản tại Đà Nẵng

In [3]:
urlDaNang = "https://batdongsan.com.vn/nha-dat-ban-da-nang/p{}"
for pagenum in range(1,108):
   try:
      browser = webdriver.Chrome(executable_path="chromedriver")
      browser.get(urlDaNang.format(pagenum))
      tagAs = browser.find_elements(By.CLASS_NAME, "js__product-link-for-product-id")
      for tagA in tagAs:
         productLinks.append(tagA.get_attribute("href"))
   except:
      continue
   browser.quit()
productLinks

['https://batdongsan.com.vn/ban-nha-mat-pho-duong-an-thuong-3-phuong-my-an/b-toa-c-ho-giua-tt-khu-di-bo-cach-bien-100m-pr36196568',
 'https://batdongsan.com.vn/ban-dat-phuong-hoa-hai-prj-fpt-city-da-nang/danh-sach-15-can-gio-hang-cuoi-nam-bat-day-gia-tot-chi-tu-2-3-ty-dien-tich-lon-pr36294963',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-pham-kiet-phuong-khue-my-prj-the-sang-residence/chao-xuan-2023-giam-tu-1-1-ty-so-huu-vinh-vien-bien-da-nang-thanh-toan-dan-trai-18-thang-pr36293749',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-pham-kiet-phuong-khue-my-prj-the-sang-residence/-san-nha-dau-nam-chinh-chu-ban-gap-lo-goc-view-bien-82m2-o-da-nang-lh-0934571866-pr36293762',
 'https://batdongsan.com.vn/ban-nha-mat-pho-duong-le-huu-trac-phuong-an-hai-dong/ban-ven-bien-da-nang-quan-son-tra-tp-da-nang-pr35884234',
 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-pham-kiet-phuong-khue-my-prj-the-sang-residence/giam-sau-24-cao-cap-gia-mua-bang-trung-cap-ch-bien-sh-lau-dai-p

Lấy các thuộc tính của bất động sản

In [4]:
detailProductsDN = []
for link in productLinks:
    try:
        browser = webdriver.Chrome(executable_path="chromedriver")
        browser.get(link)
        try:
            name = browser.find_element(By.TAG_NAME, "h1").text
            detailProduct = {"name": name}
        except:
            pass
        try:
            tagTitle = browser.find_elements(By.CLASS_NAME, "re__pr-specs-content-item-title")
            tagValue = browser.find_elements(By.CLASS_NAME, "re__pr-specs-content-item-value")
            for index in range(0, len(tagTitle)):
                detailProduct[tagTitle[index].text]= tagValue[index].text
        except:
            pass
        try:
            address = browser.find_element(By.CLASS_NAME, "re__section-body js__section-body").find_element(By.TAG_NAME, "iframe").get_attribute("data-src")
            detailProduct["Address"]= address
        except:
            pass
        detailProductsDN.append(detailProduct)
    except:
        print("link")
        continue
    browser.quit()
detailProductsDN

[{'name': 'BÁN TOÀ NHÀ CĂN HỘ GIỮA TT KHU PHỐ ĐI BỘ AN THƯỢNG, CÁCH BIỂN 100M',
  'Diện tích': '270 m²',
  'Mức giá': '72 tỷ',
  'Mặt tiền': '15 m',
  'Đường vào': '15 m',
  'Hướng nhà': 'Đông',
  'Hướng ban công': 'Đông',
  'Số tầng': '8 tầng',
  'Số phòng ngủ': '33 phòng',
  'Số toilet': '33 phòng',
  'Pháp lý': 'Chính chủ',
  'Nội thất': 'Full nội thất cơ bản - nóng lạnh'},
 {'name': 'CHÀO XUÂN 2023 GIẢM TỪ 1,1 TỶ SỞ HỮU VĨNH VIỄN CĂN HỘ BIỂN ĐÀ NẴNG, THANH TOÁN DÀN TRẢI 18 THÁNG',
  'Diện tích': '82.5 m²',
  'Mức giá': '3.487 tỷ',
  'Số phòng ngủ': '2 phòng',
  'Số toilet': '2 phòng',
  'Pháp lý': 'Sổ đỏ/ Sổ hồng'},
 {'name': 'SĂN NHÀ ĐẦU NĂM - CHÍNH CHỦ CẦN BÁN GẤP LÔ GÓC VIEW BIỂN 82M2 Ở CĂN HỘ ĐÀ NẴNG - LH: 0934 571 ***',
  'Diện tích': '82 m²',
  'Mức giá': '3.529 tỷ',
  'Hướng nhà': 'Tây - Bắc',
  'Hướng ban công': 'Đông - Nam',
  'Số phòng ngủ': '2 phòng',
  'Số toilet': '2 phòng',
  'Pháp lý': 'Sổ đỏ/ Sổ hồng',
  'Nội thất': 'Đầy đủ'},
 {'name': 'BÁN NHÀ VEN BIỂN ĐÀ NẴNG ĐƯỜ

Ghi dữ liệu qua file excel

In [6]:
csv_columns = []
for product in detailProductsDN:
    tempkhac = set(list(product)[0:]) ^ set(csv_columns)
    tempgiong = set(list(product)[0:]) & set(csv_columns)
    csv_columns = tempkhac ^ tempgiong
csv_file = "dataBatDongSanDN.csv"
try:
    with open(csv_file, 'w+',encoding= "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for dataz in detailProductsDN:
            try:
                writer.writerow(dataz)
            except:
                continue
except IOError:
    print("I/O error")